# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.
    - Numpy : A numerical computing library, its known for its execution speed, it is a C/C++ module 
    - Pandas : A panel data, work it easy to work with tabular data.dataframe is a datastructure that hold tabular data
    - Requests : usually used to make http requests
    - xlsxwriter : to save well formated documents excel document from a python script
    - math :  provide some basic math operation
    

In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). 
Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head(5)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this notebook.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this notebook, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
#from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [26]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#print(api_url)
data = requests.get(api_url)
print(type(data))
# TRANSFORM THE DATA INTO A JSON OBJECT
data = data.json()
print(type(data))

<class 'requests.models.Response'>
<class 'dict'>


In [17]:
# To check if the link works
print(data.status_code)

200


In [21]:
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ' AK REMNL)E DL/SECLQSGTNAAT(GSOAB',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'lffcioai',
 'close': None,
 'closeTime': None,
 'closeSource': 'icalioff',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 136.75,
 'latestSource': 'IEX real time price',
 'latestTime': '1:03:28 PM',
 'latestUpdate': 1666008855369,
 'latestVolume': None,
 'iexRealtimePrice': 138.36,
 'iexRealtimeSize': 305,
 'iexLastUpdated': 1652311159979,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 139.33,
 'previousVolume': 108478039,
 'change': 0.8,
 'changePercent': 0.00625,
 'volume': None,
 'iexMarketPercent': 0.017506651391433932,
 'iexVolume': 8111

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [22]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [37]:
columns = ['Tickers', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_df = pd.DataFrame(columns=columns)

In [38]:
final_df.append(
    pd.Series([symbol,price,market_cap,'N/A'],index = columns), 
    ignore_index=True
)

,Tickers,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,136.75,2318561153556,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [42]:
columns = ['Tickers', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_df = pd.DataFrame(columns=columns)
for tick in stocks['Ticker'] :
    api_url = f'https://sandbox.iexapis.com/stable/stock/{tick}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()# This instruction is very very slow so the solution will be to use a batch get instead of 1 get
    price = data['latestPrice']
    market_cap = data['marketCap']
    final_df = final_df.append(pd.Series([tick,price,market_cap,'N/A'],index = columns), ignore_index=True)
    
final_df.head(5)

    

,Tickers,Stock Price,Market Capitalization,Number of shares to buy
0,A,124.028,38365328129,N/A
1,AAL,17.086,10600663855,N/A
2,AAP,157.470,10862783171,N/A
3,AAPL,139.010,2320798958380,N/A
4,ABBV,104.090,185687737294,N/A


,Tickers,Stock Price,Market Capitalization,Number of shares to buy


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [46]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [59]:
columns = ['Tickers', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_df = pd.DataFrame(columns=columns)
ticker_groups = list(chunks(stocks['Ticker'],100))
ticker_strings = []
for ticker_group in ticker_groups:
    ticker_strings.append(','.join(ticker_group))
for ticker_string in ticker_strings :
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={ticker_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_url).json()
    for symbol in data :
        final_df = final_df.append(pd.Series([symbol, data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'],index = columns),ignore_index=True)
           
final_df

,Tickers,Stock Price,Market Capitalization,Number of shares to buy
0,A,123.650,38876634806,N/A
1,AAL,17.190,10215948032,N/A
2,AAP,157.780,10976592284,N/A
3,AAPL,138.140,2321631783822,N/A
4,ABBV,103.940,185200726172,N/A
...,...,...,...,...
500,YUM,108.450,33249782052,N/A
501,ZBH,162.701,33726880512,N/A
502,ZBRA,412.820,21957362485,N/A
503,ZION,47.260,7814925983,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [65]:
try:
    portfolio = input("Enter the size of your portfolio : ")
    val = float(portfolio)
except ValueError:
    print("Please enter a number\n Try again!")
    portfolio = input("Enter the size of your portfolio : ")
    val = float(portfolio)

position_size = val/len(final_df)
print(position_size)

Enter the size of your portfolio : 10000000
19801.980198019803


In [71]:
for i in range(len(final_df)):
    final_df.loc[i,'Number of shares to buy'] = math.floor(position_size/final_df.loc[i,'Stock Price'])
final_df

,Tickers,Stock Price,Market Capitalization,Number of shares to buy
0,A,123.650,38876634806,160
1,AAL,17.190,10215948032,1151
2,AAP,157.780,10976592284,125
3,AAPL,138.140,2321631783822,143
4,ABBV,103.940,185200726172,190
...,...,...,...,...
500,YUM,108.450,33249782052,182
501,ZBH,162.701,33726880512,121
502,ZBRA,412.820,21957362485,47
503,ZION,47.260,7814925983,419


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [82]:
writer = pd.ExcelWriter('RecommendedTrades.xlsx',engine = 'xlsxwriter')
final_df.to_excel(writer,'RecommendedTrades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [83]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format({
    'font_color':font_color,
    'bg_color' : background_color,
    'border' : 1
})

dollar_format = writer.book.add_format({
    'num_format' : '$0.00',
    'font_color':font_color,
    'bg_color' : background_color,
    'border' : 1
})

number_format = writer.book.add_format({
    'num_format' : '0',
    'font_color':font_color,
    'bg_color' : background_color,
    'border' : 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [84]:
writer.sheets['RecommendedTrades'].set_column('A:A',18,string_format)
writer.sheets['RecommendedTrades'].set_column('B:B',18,string_format)
writer.sheets['RecommendedTrades'].set_column('C:C',18,string_format)
writer.sheets['RecommendedTrades'].set_column('D:D',18,string_format)


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [85]:
writer.sheets['RecommendedTrades'].write('A1','Ticker',string_format)

0

## Saving Our Excel Output

Saving our Excel file is very easy:

In [86]:
writer.save()